In [1]:
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset
import re
import sys
sys.path.insert(0, '../src')
from dict import replacement_dict_43k

In [2]:
dataset = load_dataset("json", data_files="../data/english_pii_43k.jsonl", split="train")

In [3]:
dataset

Dataset({
    features: ['masked_text', 'unmasked_text', 'privacy_mask', 'span_labels', 'bio_labels', 'tokenised_text'],
    num_rows: 43501
})

In [4]:
print(dataset[0])

{'masked_text': "A student's assessment was found on device bearing IMEI: [PHONEIMEI_1]. The document falls under the various topics discussed in our [JOBAREA_1] curriculum. Can you please collect it?", 'unmasked_text': "A student's assessment was found on device bearing IMEI: 06-184755-866851-3. The document falls under the various topics discussed in our Optimization curriculum. Can you please collect it?", 'privacy_mask': "{'[PHONEIMEI_1]': '06-184755-866851-3', '[JOBAREA_1]': 'Optimization'}", 'span_labels': "[[0, 57, 'O'], [57, 75, 'PHONEIMEI_1'], [75, 138, 'O'], [138, 150, 'JOBAREA_1'], [150, 189, 'O']]", 'bio_labels': ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'I-PHONEIMEI', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-JOBAREA', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'tokenised_text': ['a', 'st

In [5]:
ds = pd.DataFrame.from_dict(dataset[:])
ds.head()

,masked_text,unmasked_text,privacy_mask,span_labels,bio_labels,tokenised_text
0,A student's assessment was found on device bea...,A student's assessment was found on device bea...,"{'[PHONEIMEI_1]': '06-184755-866851-3', '[JOBA...","[[0, 57, 'O'], [57, 75, 'PHONEIMEI_1'], [75, 1...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PHON...","[a, student, ', s, assessment, was, found, on,..."
1,"Dear [FIRSTNAME_1], as per our records, your l...","Dear Omer, as per our records, your license 78...","{'[FIRSTNAME_1]': 'Omer', '[VEHICLEVIN_1]': '7...","[[0, 5, 'O'], [5, 9, 'FIRSTNAME_1'], [9, 44, '...","[O, B-FIRSTNAME, I-FIRSTNAME, O, O, O, O, O, O...","[dear, om, ##er, ,, as, per, our, records, ,, ..."
2,[FIRSTNAME_1] could you please share your reco...,Kattie could you please share your recomndatio...,"{'[FIRSTNAME_1]': 'Kattie', '[AGE_1]': '72', '...","[[0, 6, 'FIRSTNAME_1'], [6, 75, 'O'], [75, 77,...","[B-FIRSTNAME, I-FIRSTNAME, O, O, O, O, O, O, O...","[kat, ##tie, could, you, please, share, your, ..."
3,Emergency supplies in [BUILDINGNUMBER_1] need ...,Emergency supplies in 16356 need a refill. Use...,"{'[BUILDINGNUMBER_1]': '16356', '[MASKEDNUMBER...","[[0, 22, 'O'], [22, 27, 'BUILDINGNUMBER_1'], [...","[O, O, O, B-BUILDINGNUMBER, I-BUILDINGNUMBER, ...","[emergency, supplies, in, 1635, ##6, need, a, ..."
4,"The [AGE_1] old child at [BUILDINGNUMBER_1], h...","The 88 old child at 5862, has showcased an unu...","{'[AGE_1]': '88', '[BUILDINGNUMBER_1]': '5862'...","[[0, 4, 'O'], [4, 6, 'AGE_1'], [6, 20, 'O'], [...","[O, B-AGE, O, O, O, B-BUILDINGNUMBER, I-BUILDI...","[the, 88, old, child, at, 58, ##6, ##2, ,, has..."


In [6]:
# new_df = []
new_df = []
document_id = 0
for row_num in range(len(ds)):
    document = document_id
    full_text = ds['unmasked_text'][row_num]
    
    tokens = ds['tokenised_text'][row_num]

    trailing_whitespace = []
    for token in tokens:
        pattern = re.escape(token.replace('##','')) + r'\s'
        if re.search(pattern, full_text, re.IGNORECASE):
            trailing_whitespace.append(True)
        else:
            trailing_whitespace.append(False)
    

    labels = ds['bio_labels'][row_num]
    for word_pos in range(len(labels)):
        word = labels[word_pos]
        # if the label exists in the dictionary, replace it
        if word in replacement_dict_43k:
            labels[word_pos] = replacement_dict_43k[word]
        else:
            labels[word_pos] = 'O'
    
    # new_df.append([row_num, full_text, tokens, trailing_whitespace, labels])
    new_df.append({'document':row_num, 'full_text':full_text, 'tokens': tokens, 'trailing_whitespace': trailing_whitespace, 'labels': labels})
    
# new_df = pd.DataFrame(new_df, columns =['document', 'full_text', 'tokens', 'trailing_whitespace', 'labels'])
# new_df = pd.DataFrame(new_df)

In [7]:
import json
with open("../data/pii_parsed_43k.json", "w") as final:
    json.dump(new_df, final)

In [8]:
# ds = Dataset.from_pandas(new_df, split = 'train+validation')
# ds
# ds.save_to_disk("../data/")
# new_df.to_json('../data/pii_parsed_43k.json')